Concrete_cracks_identifer

This code was done in colab so tweak appropriately for jupyter ;follow notes on creating appropriate folder structure 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
##### kaggle dataset https://www.kaggle.com/datasets/arnavr10880/concrete-crack-images-for-classification
### Referenced an exercise I completed on kaggle https://www.kaggle.com/code/evansataforson/exercise-custom-convnets/edit
import zipfile

# Specify the file path and extraction directory (replace with your actual paths)
file_path = '/content/drive/MyDrive/ML_DATA/concrete_crack_data/archive (1).zip'
#file_path = '/content/archive (1).zip' #use this when working locally the archive file can be got from the dataset link
extract_dir = '/content/concrete_images'

try:
  # Attempt to extract the zip file
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
  print(f"Extracted files from '{file_path}' to '{extract_dir}' successfully.")
except Exception as e:
  # Handle potential errors during extraction
  print(f"Error occurred during extraction: {e}")



  #########

#### splitting data into folders

import os
from shutil import move

# Define base directory (assuming you mounted your drive)
#base_dir = "/content/concrete_images"  # Replace with your base directory if mounted

base_dir = "/content"
# Define subfolder names (modify if needed)
data_dir = "concrete_images"  # Assuming data is in a subfolder within the base directory
train_dir = "train"
val_dir = "validation"
test_dir = "test"

# Define class folders within data directory (assuming only Negative and Positive)
class_folders = ["Negative", "Positive"]

# Define the split ratio (e.g., 70% train, 15% validation, 15% test--- playing with 0.05. 0.02 and 0.93)
#train_ratio = 0.7
#val_ratio = 0.15
#test_ratio = 0.15

# Define the split ratio (--- playing with 0.05. 0.02 and 0.93 so it runs fast in colab the above setting should be used)
train_ratio = 0.05
val_ratio = 0.02
test_ratio = 0.93


def split_data(class_folder):
  # Get all image file paths for the class
  class_path = os.path.join(base_dir, data_dir, class_folder)
  all_files = [os.path.join(class_path, f) for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

  # Split the data into train, validation, and test sets
  num_files = len(all_files)
  num_train = int(num_files * train_ratio)
  num_val = int(num_files * val_ratio)
  num_test = int(num_files * test_ratio)
  train_files, val_files, test_files = all_files[:num_train], all_files[num_train:num_train+num_val], all_files[num_train+num_val:]

  # Create train, validation, and test directories for the class within data_dir
  os.makedirs(os.path.join(base_dir, data_dir, train_dir, class_folder), exist_ok=True)
  os.makedirs(os.path.join(base_dir, data_dir, val_dir, class_folder), exist_ok=True)
  os.makedirs(os.path.join(base_dir, data_dir, test_dir, class_folder), exist_ok=True)

  # Move images to respective directories
  for file in train_files:
    move(file, os.path.join(base_dir, data_dir, train_dir, class_folder))

  for file in val_files:
    move(file, os.path.join(base_dir, data_dir, val_dir, class_folder))

  for file in test_files:
    move(file, os.path.join(base_dir, data_dir, test_dir, class_folder))

  print(f"Successfully split {class_folder} images into train, validation, and test sets.")

# Loop through each class folder and perform splitting
for class_folder in class_folders:
  split_data(class_folder)

Setting up loading

In [ ]:
################set up
 # Setup feedback system
#from learntools.core import binder
#binder.bind(globals())
#from learntools.computer_vision.ex5 import *

# Imports
#import os, warnings
import warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells


# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    '/content/concrete_images/train',
    labels='inferred',
    label_mode='binary',
    image_size=[227, 227],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    '/content/concrete_images/validation',
    labels='inferred',
    label_mode='binary',
    image_size=[227, 227],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)



Defining Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    # Block One
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                  input_shape=[227, 227, 3]),
    layers.MaxPool2D(),

    # Block Two
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Three
    # YOUR CODE HERE
    layers.Conv2D(filters=227, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=227, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
])

Compile

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)


Test Perfomance

In [ ]:
# epochs=50, should be used instead of 5 which is used to make it run faster in colab
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=5,
)

In [ ]:
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

Test prediction

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Load a test image
test_image_path = '/content/concrete_images/test/Positive/00007.jpg'
img = load_img(test_image_path, target_size=(227, 227))

# Preprocess the image (replace with your specific preprocessing steps)
test_image = img_to_array(img)
#test_image = test_image / 255.0  # Normalize pixel values (assuming this was done during training)
test_image = np.expand_dims(test_image, axis=0)

# Predict on the test image
predictions = model.predict(test_image)

# Interpret predictions for classification (assuming model predicts class probabilities)
predicted_class = np.argmax(predictions[0])  # Get index of class with highest probability
#class_names = ['Positive', 'Negative']  # Replace with your actual class names (if applicable)

# Display the image
plt.imshow(img)
plt.axis('off')  # Hide axes for a cleaner presentation (optional)
plt.show()


print(f"Predicted class: {class_folders[predicted_class]}")

In [ ]:
Save Model to google drive

In [ ]:
#from google.colab import drive

# Mount your Google Drive (replace with authentication steps if needed)
#drive.mount('/content/gdrive')

# Define the path to save the model within your Google Drive (replace with your desired location)
#model_path = "/content/gdrive/My Drive/concrete_images/saved_models"
model_path ="/content/drive/MyDrive/ML_DATA/concrete_crack_data/saved_models"
os.makedirs(model_path, exist_ok=True)  # Create directory if it doesn't exist (optional)

# Save the model using tf.keras.models.save
model.save(model_path, save_format="tf")

# Get the base filename (without directory path) of the saved model
saved_model_filename = os.path.basename(model_path)

# Print the saved model name
print(f"Model saved successfully to Google Drive: {saved_model_filename}")

In [ ]:
Load model from Google drive and test

In [ ]:
#from google.colab import drive
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
# Mount your Google Drive (replace with authentication steps if needed)
#drive.mount('/content/gdrive')

# Define the path where you saved the model within your Google Drive
model_path = "/content/drive/MyDrive/ML_DATA/concrete_crack_data/saved_models"  # Replace with your path

# Load the saved model from Google Drive
loaded_model = load_model(model_path)

# Print confirmation message
print("Model loaded successfully from Google Drive!")



# Load a test image
test_image_path_local = '/content/concrete_images/test/Positive/00007.jpg'
img = load_img(test_image_path_local, target_size=(227, 227))

# Preprocess the image (replace with your specific preprocessing steps)
test_image_val = img_to_array(img)
#test_image = test_image / 255.0  # Normalize pixel values (assuming this was done during training)
test_image_val = np.expand_dims(test_image_val, axis=0)


predictions = loaded_model.predict(test_image_val)

# Interpret predictions for classification (assuming model predicts class probabilities)
predicted_class = np.argmax(predictions[0])  # Get index of class with highest probability

# Display the image
plt.imshow(img)
plt.axis('off')  # Hide axes for a cleaner presentation (optional)
plt.show()


print(f"Predicted class: {class_folders[predicted_class]}")